<a href="https://colab.research.google.com/github/hongo-daisuke/image-identification-app/blob/master/make_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# モデルの訓練と保存

Google Colaboratoryで画像認識用のモデルを構築し、訓練を行う。\
今回は、cifar-10を訓練データに使用する。

In [ ]:
import tensorflow
import keras
print(tensorflow.__version__)
print(keras.__version__)

# 訓練データの読み込み
cifar-10を読み込む

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator

(x_train, t_train), (x_test, t_test) = cifar10.load_data()

# モデルの構築

CNNのモデルを構築

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import adam_v2
from tensorflow.keras import utils as np_utils

batch_size = 32
epochs = 20
n_class = 10

t_train = np_utils.to_categorical(t_train, n_class)  # one-hot表現に
t_test = np_utils.to_categorical(t_test, n_class)


model = Sequential()

model.add(Conv2D(32, (3, 3), padding="same", input_shape=x_train.shape[1:]))  # ゼロパディング、バッチサイズ以外の画像の形状を指定
model.add(Activation("relu"))
model.add(Conv2D(32, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # 一次元の配列に変換
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))  # ドロップアウト
model.add(Dense(n_class))
model.add(Activation("softmax"))

model.compile(optimizer=adam_v2.Adam(), loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

## 学習
モデルの訓練を行う。  
ImageDataGeneratorを使用する際は、学習にfit_generatorメソッドを使用。  

In [ ]:
x_train = x_train / 255  # 0から1の範囲に収める
x_test = x_test / 255

generator = ImageDataGenerator(
           rotation_range=0.2,
           width_shift_range=0.2,
           height_shift_range=0.2,
           shear_range=10,
           zoom_range=0.2,
           horizontal_flip=True)
generator.fit(x_train)

history = model.fit_generator(generator.flow(x_train, t_train, batch_size=batch_size),
                              epochs=epochs,
                              validation_data=(x_test, t_test))

## 学習の推移
学習の推移を表示。  

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 

train_loss = history.history["loss"]  # 訓練用データの誤差
train_acc = history.history["accuracy"]  # 訓練用データの精度
val_loss = history.history["val_loss"]  # 検証用データの誤差
val_acc = history.history["val_accuracy"]  # 検証用データの精度

plt.plot(np.arange(len(train_loss)), train_loss, label="loss")
plt.plot(np.arange(len(val_loss)), val_loss, label="val_loss")
plt.legend()
plt.show()

plt.plot(np.arange(len(train_acc)), train_acc, label="accuracy")
plt.plot(np.arange(len(val_acc)), val_acc, label="val_acc")
plt.legend()
plt.show()

## 評価
モデルの評価を行う。  

In [ ]:
loss, accuracy = model.evaluate(x_test, t_test)
print(loss, accuracy)

# モデルの保存とダウンロード

他の環境でも使用出来るように、モデルをダウンロードする。

In [ ]:
from google.colab import files

model.save("image_classifier.h5",  include_optimizer=False)
files.download("image_classifier.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>